In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [2]:
default_config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 100
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 50,
                    "group_penalty": 1,
                    "shift_penalty": 1
                }
            },
            "simulated_annealing": {
                "initial_temperature": 100,
                "cooling_rate": 0.1,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "algorithm": "genetic_algorithm",
        "config": {
            "max_iteration": 100,
            "population_size": 25,
            "elitism_size": 2,
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 10,
                    "group_penalty": 1,
                    "shift_penalty": 1
                }
            },
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": False,
                    "elitism": True,
                    "tournament_size": 5
                },
                "crossover": {
                    "single_point": True,
                    "two_point": False,
                    "uniform": False,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": False,
                    "mutation_probability": 0.1
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}




In [3]:
from scheduling_algorithm.data_parser import ModuleData
from scheduling_algorithm.algorithms import (
    GeneticAlgorithm,
    GeneticLocalSearch
)
from scheduling_algorithm.utils.solution_generator import SolutionGenerator
from scheduling_algorithm.factory import WeeklyFactory

In [4]:
modules = ModuleData.get_modules_by_semester(1)

In [5]:
generator = SolutionGenerator.from_data(default_config)
weekly_generator = SolutionGenerator.from_data(default_config)

Loading configuration...
Configuration loaded successfully.
Creating Genetic Algorithm Object from Configuration File
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Elitism:  Selection(name=ElitismSelection)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=ElitismSelection)]
Configuring crossover operator:  [Crossover(name=SinglePointCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]
Loading configuration...
Configuration loaded successfully.
Creating Genetic Algorithm Object from Configuration File
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Elitism:  Selection(name=ElitismSelection)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=Elitism

In [6]:
solution = weekly_generator.generate_solution_weekly()

Generating population for module 1 week 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module: 1
Repairing weekly, week: 1, module

In [7]:
solution

Chromosome(length=288, fitness=6)

In [ ]:
from math import ceil
def calculate_module_weeks(module_id):
    module_date = ModuleData.get_dates(module_id)
    start_date = module_date.start_date
    end_date = module_date.end_date
    weeks = ceil((end_date - start_date).days / 7)
    return weeks

In [ ]:
from scheduling_algorithm.structure.chromosome import Chromosome

best_chromosome = Chromosome()
for module in modules:
    num_weeks = calculate_module_weeks(module.id)
    for week in range(num_weeks):
        factory_instance = WeeklyFactory(weeks=num_weeks, week= week + 1, module_id=module.id)
        print(f"Generating population for module {module.id} week {week + 1}")
        weekly_population = factory_instance.generate_population(population_size=25)
        if len(weekly_population) == 0:
            print(f"Module {module.id} week {week + 1} has no population, all the remaining chapter are already assigned on previous weeks")
            print("Skipping to next module")
            break
        weekly_chromosome = generator.algorithm.run(population=weekly_population)
        best_chromosome += weekly_chromosome

In [ ]:
best_chromosome